In [1]:
from pathlib import Path
from object_summary import clf_folders_to_df, objects_in_categories, dump_to_json_file, read_json_from_file
from tf_object_detection_util.inference_api import TFInference

/home/prasannals/anaconda3/envs/tf_obj/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/prasannals/anaconda3/envs/tf_obj/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/prasannals/anaconda3/envs/tf_obj/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/prasannals/anaconda3/envs/tf

In [2]:
path = Path('data/messy_vs_clean/sample/')
out_path = Path('out/messy_or_clean/sample/')
out_path.mkdir(exist_ok=True)

def ls(path): return [p for p in path.glob('*')]

"path" variable is the directory containing a list of folders where each folder contains all images of a single category and the name of the folder is the label for the category.

"out_path" is the path where the output visualizations (if visualize=True in "objects_in_categories" method) are stored.

In [3]:
# here, we're using a small subset from Messy vs Clean dataset from Kaggle
# https://www.kaggle.com/cdawn1/messy-vs-clean-room
ls(path)

[PosixPath('data/messy_vs_clean/sample/messy'),
 PosixPath('data/messy_vs_clean/sample/clean')]

We need to pass the path to the ".pb" file (this is the inference graph. the model we use to perform object detection) and the ".pbtxt" file (this provides the mapping from the categorical output of the model (say 0, 1, 2) to the labels of the corresponding objects (say "cat", "dog", "horse")) into the TFInference class. We then need to pass the created TFInference object into the "objects_in_categories" to perform the summarization.

We need to download the pretrained object detection model from tensorflow. Use the below link to download a Faster RCNN model trained on [Open Images](https://opensource.google/projects/open-images-dataset) - 

pb file contained in -  http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12.tar.gz

pbtxt file - https://github.com/tensorflow/models/blob/master/research/object_detection/data/oid_v4_label_map.pbtxt

To select other object detection models, go to the following link - 

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md


In [4]:
def load_faster_rcnn_open_images():
    MODEL_NAME = Path('/home/prasannals/models/research/object_detection/faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12')
    PATH_TO_FROZEN_GRAPH = MODEL_NAME / 'frozen_inference_graph.pb'
    PATH_TO_LABELS = MODEL_NAME / 'oid_v4_label_map.pbtxt'

    inf = TFInference(PATH_TO_FROZEN_GRAPH, PATH_TO_LABELS)
    return inf

In [5]:
# clf_folders_to_df converts the input files into the required format in a DataFrame.
# the output of the "clf_folders_to_df" will be the input to "objects_in_categories" function
path_df = clf_folders_to_df(path)
inf = load_faster_rcnn_open_images()

In [6]:
### OPTIONAL - split your "path_df" into several equal DataFrames 
### (useful if you intend to parallely run the code on different machines/GPUs)
from object_summary.util import split_df

split_df(path_df, 2)

[                                                path category
 0  /home/prasannals/object_detection/data/messy_v...    messy
 1  /home/prasannals/object_detection/data/messy_v...    messy,
                                                 path category
 2  /home/prasannals/object_detection/data/messy_v...    clean
 3  /home/prasannals/object_detection/data/messy_v...    clean]

In [7]:
path_df

,path,category
0,/home/prasannals/object_detection/data/messy_v...,messy
1,/home/prasannals/object_detection/data/messy_v...,messy
2,/home/prasannals/object_detection/data/messy_v...,clean
3,/home/prasannals/object_detection/data/messy_v...,clean


In [8]:
res = objects_in_categories(path_df, inf, out_path, visualize=True)

In [9]:
res

[{'num_detections': 3,
  'detection_boxes': array([[0.4943506 , 0.02411438, 0.97711736, 0.98587483],
         [0.55433404, 0.5395526 , 0.95111483, 1.        ],
         [0.        , 0.        , 0.97139925, 1.        ]], dtype=float32),
  'detection_scores': array([0.5345746 , 0.40630788, 0.3590906 ], dtype=float32),
  'detection_classes': array([281, 281, 241]),
  'detection_classes_translated': ['Table', 'Table', 'House'],
  'category': 'messy'},
 {'num_detections': 7,
  'detection_boxes': array([[0.7609224 , 0.31506044, 0.8543164 , 0.51453626],
         [0.7031163 , 0.7674436 , 0.9546674 , 0.97680485],
         [0.39719838, 0.39140877, 0.78039163, 0.87248987],
         [0.6339927 , 0.4408361 , 0.718982  , 0.5389727 ],
         [0.3982816 , 0.3679562 , 0.78282356, 0.8985413 ],
         [0.18970141, 0.41172495, 0.6037266 , 0.9957504 ],
         [0.22396113, 0.00430926, 0.7286406 , 0.460847  ]], dtype=float32),
  'detection_scores': array([0.7762132 , 0.63690084, 0.6345127 , 0.6321289 ,

In [10]:
dump_to_json_file(res, Path('out/messy_or_clean_res_sample.json'))